## Model Evaluation 

In [1]:
import os

In [2]:
os.chdir("../")

In [3]:
%pwd

'c:\\Users\\VenuraP\\Desktop\\Browns Data Projects\\ML Projects\\POC\\Harti-Food-Price-Prediction'

## Entity

In [4]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelEvaluationConfig:
    root_dir: Path
    model_path: Path
    metric_file_name: Path


In [5]:
from src.constants import *
from src.utils.common import read_yaml

## Configuration  Manager

In [6]:
# Function to initialize configuration by reading YAML and creating directories
def load_configuration(config_filepath: Path = CONFIG_FILE_PATH, schema_filepath: Path = SCHEMA_FILE_PATH):

    config = read_yaml(config_filepath)
    schema = read_yaml(schema_filepath)

    return config, schema

# Function to get data ingestion configuration from the loaded config
def get_model_evaluation_config(config) -> ModelEvaluationConfig:

    # Extract data ingestion settings from the config
    model_evaluation = config.model_evaluation

    # Create and return a DataIngestionConfig instance
    return ModelEvaluationConfig(
        root_dir=model_evaluation.root_dir,
        model_path=model_evaluation.model_path,
        metric_file_name=model_evaluation.metric_file_name
    )

## Model Evaluation Component

In [7]:
import pandas as pd
import numpy as np
import tensorflow as tf
from datetime import datetime
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

def model_evaluation(train_x, test_x, train_y, test_y,config):

    # Load the model
    best_model = tf.keras.models.load_model(config.model_path)

    # Predict on train data
    predicted_train_set_y = best_model.predict(train_x)

    # Predict on test data
    predicted_test_set_y = best_model.predict(test_x)

    # Calculate evaluation metrics for the training data
    train_r2 = r2_score(train_y, predicted_train_set_y)
    train_mae = mean_absolute_error(train_y, predicted_train_set_y)
    train_mse = mean_squared_error(train_y, predicted_train_set_y)
    train_rmse = np.sqrt(train_mse)

    # Calculate evaluation metrics for the test data
    test_r2 = r2_score(test_y, predicted_test_set_y)
    test_mae = mean_absolute_error(test_y, predicted_test_set_y)
    test_mse = mean_squared_error(test_y, predicted_test_set_y)
    test_rmse = np.sqrt(test_mse)

    # Get current date and time
    current_time = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

    # Prepare data for saving
    metrics_data = {
        "Date": [current_time],  
        "Train R2": [train_r2],
        "Train MAE": [train_mae],
        "Train MSE": [train_mse],
        "Train RMSE": [train_rmse],
        "Test R2": [test_r2],
        "Test MAE": [test_mae],
        "Test MSE": [test_mse],
        "Test RMSE": [test_rmse]
    }

    # Convert to DataFrame
    metrics_df = pd.DataFrame(metrics_data)

    # Save metrics to Excel (appending to the file if it exists)
    excel_file = config.metric_file_name
    try:
        # If the Excel file already exists, append the new data
        with pd.ExcelWriter(excel_file, engine='openpyxl', mode='a', if_sheet_exists='overlay') as writer:
            metrics_df.to_excel(writer, sheet_name='Metrics', index=False, header=False, startrow=writer.sheets['Metrics'].max_row)
    except FileNotFoundError:
        # If the file doesn't exist, create a new file and write data
        with pd.ExcelWriter(excel_file, engine='openpyxl') as writer:
            metrics_df.to_excel(writer, sheet_name='Metrics', index=False)

## Model Evaluation Pipeline

In [8]:
from src import logger
from src.configuration.configuration import get_model_trainer_config
from src.components.model_trainer import sequence_creation_train_test_split
from src.utils.common import create_directories
from src.configuration.configuration import load_configuration

def model_trainer_training_pipeline():
    
    try:
        # Load config and schema
        config, schema = load_configuration()

        model_trainer_config = get_model_trainer_config(config,schema) 

        # Retrieve the model trainer configuration from the loaded config
        model_evaluation_config = get_model_evaluation_config(config)

        # Create directories related to model evaluation (root directory)
        create_directories([model_evaluation_config.root_dir])

        train_x, test_x, train_y, test_y = sequence_creation_train_test_split(model_trainer_config)

        # model evaluation
        model_evaluation(train_x, test_x, train_y, test_y,model_evaluation_config)

    except Exception as e:
        logger.error(f"An error occurred during model evaluation: {e}")

if __name__ == "__main__":

 model_trainer_training_pipeline()

[2024-10-05 01:55:37,092: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-10-05 01:55:37,092: INFO: common: yaml file: schema.yaml loaded successfully]
[2024-10-05 01:55:37,098: INFO: common: created directory at: artifacts/model_evaluation]
45/45 ━━━━━━━━━━━━━━━━━━━━ 3s 42ms/step
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
